In [13]:
import csv
proteins = []
with open("C:\\Users\\60553\\Downloads\\Diease-related Proteins, ChatGPT.csv", 'r', newline='',encoding = 'utf-8') as f:
            

    rows = csv.reader(_.replace('\x00', '') for _ in f)
    next(rows)

    # Get Column names
    keys = next(rows)
    # Add values by column
    for row in rows:
        if row[4] != "":
            proteins.append(row[4])

In [17]:
len(proteins)

211

In [30]:
import requests
import json
import os
def download_protein(uniprot_entry):
    if os.path.exists(f"D:\\PyCharm\\pythonProject\\Sciwing\\AI4BioMed\\data\\{uniprot_entry}.json"):
        print(f"{uniprot_entry} Existed.")
        return
    url = f"https://www.uniprot.org/uniprot/{uniprot_entry}.json"
    response = requests.get(url)
    data = response.json()
    with open(f"D:\\PyCharm\\pythonProject\\Sciwing\\AI4BioMed\\data\\{uniprot_entry}.json","w") as f:
        data = json.dumps(data)
        f.write(data)
    print(f"{uniprot_entry}.")

In [31]:
for entry in proteins:
    download_protein(entry)

P00747 Existed.
P15692 Existed.
P14780 Existed.
P01375 Existed.
Q99836 Existed.
P01033 Existed.
P09429 Existed.
P06702 Existed.
P49913 Existed.
P01033 Existed.
P35916 Existed.
O14672 Existed.
P35968 Existed.
O15263 Existed.
O15263 Existed.
Q15109 Existed.
O95831 Existed.
P03950 Existed.
P59666 Existed.
O00622 Existed.
Q9Y275 Existed.
P06213 Existed.
P15692 Existed.
P61073 Existed.
Q12866 Existed.
P19320 Existed.
P08631 Existed.
P14780 Existed.
O43927 Existed.
P22894 Existed.
P09429 Existed.
P49238 Existed.
P08253 Existed.
P00797 Existed.
P19320 Existed.
P14735 Existed.
P03973 Existed.
P08253 Existed.
P58753 Existed.
P39059 Existed.
Q02763 Existed.
Q9NRM6 Existed.
Q8WTQ1 Existed.
O00622 Existed.
Q16552 Existed.
P01116 Existed.
P09238 Existed.
P01024 Existed.
P27986 Existed.
Q13686 Existed.
P09455 Existed.
P05154 Existed.
P29508 Existed.
O14672 Existed.
Q92583 Existed.
P98155 Existed.
P24821 Existed.
P23297 Existed.
O00258 Existed.
O15382 Existed.
P29692 Existed.
Q9Y243 Existed.
Q9GZX6 E

In [1]:
def get_input(protein_entry):

    res = {}
    with open(f"D:\\PyCharm\\pythonProject\\Sciwing\\AI4BioMed\\data\\{protein_entry}.json","r") as f:
        string = f.readlines()[0]
        entry = json.loads(string)

        comments_text = ""
        comments_text_with_keys = ""
        
        ref_text = ""
        uniProtKB_text = ""
        
        input_texts = ""
        input_texts_with_keys = ""
        # comments value
        active_comments = {"FUNCTION":[], "CATALYTIC ACTIVITY":[], "COFACTOR":[], "ACTIVITY REGULATION":[],
                       "TISSUE SPECIFICITY":[], "INDUCTION":[], "DOMAIN":[], "DISEASE":[]}

        if "comments" in entry.keys():
            comments = entry["comments"]
            for comment in comments:
                value = ""
                if "commentType" in comment.keys() and comment["commentType"] in active_comments.keys():
                    if comment["commentType"] == "CATALYTIC ACTIVITY":
                        value = comment["reaction"]["name"]
                    elif comment["commentType"] == "COFACTOR":
                        if "note" in comment.keys():
                            value = comment["note"]["texts"][0]["value"]
                    elif comment["commentType"] == "DISEASE":
                        values = []
                        if "disease" in comment.keys():
                            values.append(comment["disease"]["description"])
                        if "note" in comment.keys():
                            values.append(comment["note"]["texts"][0]["value"])
                        value = " ".join(values)
                    else:
                        value = comment["texts"][0]["value"]
                    active_comments[comment["commentType"]].append(value)
            for k, v in active_comments.items():
                comments_text += " ".join(v)
                t = " ".join(v)
                comments_text_with_keys += (k + ": " + t)
        # references title
        if "references" in entry.keys():
            references = entry["references"]
            titles = []
            for ref in references:
                if "title" in ref["citation"].keys():
                    titles.append(ref["citation"]["title"])
            ref_text = " ".join(titles)

        # uniProtKBCrossRef
        if "uniProtKBCrossReferences" in entry.keys():
            uniProt_references = entry["uniProtKBCrossReferences"]
            uniProt_des = []
            for ref in uniProt_references:
                properties = {}
                for item in ref["properties"]:
                    properties[item["key"]] = item["value"]
                if "Description" in properties.keys():
#                     print(properties["Description"])
                    if properties["Description"] != "-":
                        uniProt_des.append(properties["Description"])
            uniProtKB_text = " ".join(uniProt_des)

        # input
        input_texts = comments_text + ref_text + uniProtKB_text
        res["protein"] = protein_entry
        res["input_text"] = input_texts
        res["input_text_with_keys"] = comments_text + "Reference titles: " + ref_text 
#         res["comments"] = comments_text
#         res["ref_titles"] = ref_text
#         res["uniProt_description"] = uniProtKB_text
    return res


    

In [14]:
labels = {}
with open("C:\\Users\\60553\\Downloads\\Diease-related Proteins, ChatGPT.csv", 'r', newline='',encoding = 'utf-8') as f:
            

    rows = csv.reader(_.replace('\x00', '') for _ in f)
    next(rows)

    # Get Column names
    keys = next(rows)
    # Add values by column
    for row in rows:
        if row[4] != "":
            label = (row[10], row[11])
            labels[row[4]] = label
            


In [18]:
dataset = []
for i in proteins:
    res = get_input(i)
    data = res
    if i in labels.keys():
        data["labels"] = "Score: " + labels[i][0] + "."
        if len(labels[i][1])>1:
            data["labels"] += " Explanation: " + labels[i][1]
        data["Score"] = labels[i][0]
        data["Explanation"] = labels[i][1]
        dataset.append(data)

In [19]:
with open(f"D:\\PyCharm\\pythonProject\\Sciwing\\AI4BioMed\\uniprot.json","w") as f:
    json.dump(dataset, f, indent = 4)
    
